In [3]:
import gspread
import pandas as pd
import os
from IPython.core.display import HTML
from oauth2client.client import SignedJwtAssertionCredentials
import json


In [4]:
json_key = json.load(open("/home/gpratt/ipython_notebook/public clip-588adbc137f3.json"))
scope = ['https://spreadsheets.google.com/feeds']

credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
gc = gspread.authorize(credentials)

sht1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0")
ws = sht1.worksheet("public_clip_database")
list_of_lists = ws.get_all_values()
df = pd.DataFrame(list_of_lists[1:], columns=list_of_lists[0])
df.index = df.SRA

clip_df = df[df['CLIP-seq?'] == "Yes"]
clip_df = clip_df[clip_df.type != "iclip"]
clip_df['file_names'] = clip_df.SRA.apply(lambda x: os.path.join("/home/gpratt/projects/public_clip/analysis/public_clip_v4", x.strip()) + ".fastq.gz")

In [5]:
#manifest
clip_df.to_csv("/home/gpratt/projects/public_clip/scripts/public_fastq.txt", columns=['file_names', 'Species', 'replicate'], sep="\t", index=False, header=False)

In [6]:
#demux sra files
result_file = []
with open("/home/gpratt/projects/public_clip/scripts/sra_files.txt", 'w') as out_file:
    for root, subFolders, files in os.walk("/projects/ps-yeolab/clip_public/clip/"):
        for fn in files:
            if fn.endswith("sra"):
                result_file.append(os.path.abspath(os.path.join(root, fn)))

In [7]:
cmd_list = []

with open("/home/gpratt/projects/public_clip/scripts/fastq_dump_clip.sh", 'w') as out_file:

    out_file.write("""#!/bin/bash                                                                                                                                           
#PBS -N fastq_dump_clip                                                                                                                                       
#PBS -o fastq_dump_clip.sh.out                                                                                                                                
#PBS -e fastq_dump_clip.sh.err                                                                                                                                
#PBS -V                                                                                                                                                       
#PBS -l walltime=4:00:00                                                                                                                                      
#PBS -l nodes=1:ppn=1                                                                                                                                         
#PBS -A yeo-group                                                                                                                                             
#PBS -q home-yeo                                                                                                                                                 
#PBS -t 1-335                                                                                                                                                  
# Go to the directory from which the script was called                                                                                                         
cd /home/gpratt/projects/public_clip/analysis/public_clip_v4
""")

    for line in result_file:
            line = line.strip()
            sra_name = os.path.basename(line).split(".")[0]
            cmd_list.append('fastq-dump --gzip %s' % (line))

    for x,  line in enumerate(cmd_list):
            out_file.write("cmd[{}]=\"{}\"\n".format(x, line))

    out_file.write("eval ${cmd[$PBS_ARRAYID]}\n")
